### 1. Load libraries

In [1]:
import pandas as pd

In [2]:
from scoremtb.score import compute_score as cs
from scoremtb.util import combine as comb
from scoremtb.score import get_syndata as gsyn

### 2. Load dataset

In [3]:
PROJECT_ID = 'syn29357579'

In [4]:
#Load data from S3
# syn = gsyn.get_synclient()
# dfid = gsyn.get_data(syn, PROJECT_ID, 'dataset_archivemetadata_v1/')
# stepdata = gsyn.get_data(syn, PROJECT_ID, 'dataset_sharedschema_v1_steps/')
# task_data = gsyn.get_data(syn, PROJECT_ID, 'dataset_sharedschema_v1/')
# task_status = gsyn.get_data(syn, PROJECT_ID, 'dataset_sharedschema_v1_taskStatus/')

######## OR ###########
#Load data from downloaded csv's
dfid = pd.read_csv('../../MTB/MTB_scoring/data/dfid.csv', low_memory=False)
stepdata = pd.read_csv('../../MTB/MTB_scoring/data/stepdata.csv', low_memory=False)
task_data = pd.read_csv('../../MTB/MTB_scoring/data/task_data.csv', low_memory=False)
task_status = pd.read_csv('../../MTB/MTB_scoring/data/task_status.csv', low_memory=False)

In [5]:
#Merge task status
task_status = task_status.drop(columns=['id', 'index'])
dfid = dfid.merge(task_status, on = ['recordid', 'assessmentid', 'year', 'month', 'day'],
                    how='left').reset_index(drop=True)

### 3. Filter Scores

##### 3.1 Study membership details

In [6]:
def study_membership():
    syn = gsyn.get_synclient('USER_NAME','PASSWORD') 
    
    table_syn = syn.tableQuery('SELECT * FROM syn29300177')
    study_details = table_syn.asDataFrame()
    study_details = study_details.sort_values(by=['healthCode', 'participantVersion']).reset_index(drop=True)
    
    study_details = study_details[['healthCode','studyMemberships']]
    study_details = study_details.rename(columns={"healthCode": "healthcode"})
    
    study_details = study_details.drop_duplicates(keep='last')
    return study_details.reset_index(drop=True)

In [23]:
study_reference = study_membership()

#### 3.2 Spelling

In [8]:
spelling_score = cs.score_spell_vocab_psm(task_data, dfid, 'Spelling', study_reference)

#### 3.3 Vocab

In [9]:
vocab_score = cs.score_spell_vocab_psm(task_data, dfid, 'Vocab', study_reference)

#### 3.4 PSM

In [10]:
PSM_score = cs.score_spell_vocab_psm(task_data, dfid, 'PSM', study_reference)

[WARNING] /home/ssm-user/Vijay/MTB_testing/MTB_scoring/scoremtb/score/get_spellvocab_score.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  task_scrd[config['psm_adj']] = task_scrd.loc[:, [config['scores_adjacentP1'], config['scores_adjacentP2']]].sum(axis=1)

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  task_scrd[config['psm_adj']] = task_scrd.loc[:, [config['scores_adjacentP1'], config['scores_adjacentP2']]].sum(axis=1)

[WARNING] /home/ssm-user/Vijay/MTB_testing/MTB_scoring/scoremtb/score/get_spellvocab_score.py:57: Setting

#### 3.5 MFS

In [11]:
score_mfs = cs.score_event_common(task_data, stepdata, dfid, 'MFS', study_reference)

#### 3.6 DCCS

In [12]:
score_dccs = cs.score_event_common(task_data, stepdata, dfid, 'DCCS', study_reference)

#### 3.7 Fname

In [13]:
score_fname = cs.score_event_common(task_data, stepdata, dfid, 'FNAME', study_reference)

#### 3.8 Number Match

In [14]:
score_nm = cs.score_event_common(task_data, stepdata, dfid, 'Number Match', study_reference)

#### 3.9 Flanker

In [15]:
score_flanker = cs.score_event_common(task_data, stepdata, dfid, 'Flanker', study_reference)

### 4. Combine Score

In [16]:
stack_merged = comb.combine_scores(score_mfs, score_dccs, score_fname, score_nm, score_flanker, PSM_score, 
                                   spelling_score, vocab_score, dfid)

In [19]:
# stack_merged.to_csv('MTB_UCSF_scores.csv', index=False)